In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import investpy
import datetime

In [0]:
# Obtention de la date du jour, format texte, pour paramètre date_max investpy
current_time = datetime.datetime.now()

date_jour = str(current_time.day) + "/" + str(current_time.month) + "/" + str(current_time.year)
print("Date du jour : ",date_jour)

Le 15 Juin 2006 est identifié comme la première date intégrant les données de volume

In [0]:
df_W_historical_data = investpy.indices.get_index_historical_data(
    index="Nasdaq 100",
    country="united states",
    from_date="15/06/2006",
    to_date=date_jour,
    interval="Weekly"
)

In [0]:
# Par défaut la colonne date est un index.
# Passage de celle-ci en colonne
df_W_historical_data.reset_index(inplace=True)

In [0]:
df_W_historical_data.drop(columns="Currency", inplace=True)

In [0]:
# Test si aucune ligne manquante
test_list = [champ == 0 for champ in df_W_historical_data.isnull().sum()]
# Si toutes les colonnes sont True, résultat = True
res1 = all(i for i in test_list)

In [0]:
# Test si toutes les colonnes > 0
''' Pour l'hebdo on ne tiendra pas nécessairement compte du volume
Celui-ci est rempli avec > 1 mois de retard sur investing.com
Par ailleurs son rôle de Leading indicator ne semble pas se vérifier en Hebdo.
Remarque : Etude du MFI en leading indicator sur Timeframe plus faible que Daily ?'''

test_list_zero = [champ > 0 for champ in df_W_historical_data[["Open","High","Low","Close"]].describe().min()]
res2 = all(z for z in test_list_zero)

In [0]:
# Write recipe outputs
nasdaq_100_weekly_historical_data = dataiku.Dataset("investpy_ndx_hist_2006_weekly")

if res1 & res2 :
    nasdaq_100_weekly_historical_data.write_with_schema(df_W_historical_data)
    print("Import réussi et tests validés")
    print("Nombre de lignes : ",df_W_historical_data.shape[0])
else :
    print("Au moins une ligne est vide ou une valeur est incohérente.\nPas d'écriture")